In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base-v2")


e:\Code Summary\summarize-torch\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from transformers import BertModel, BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM
config = BertConfig.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
model = BertModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

inputs = tokenizer(["Tôi đang học ở trường đại học tự nhiên", "Tôi là ai"], return_tensors="pt", padding=True, max_length=max)
inputs2 = tokenizer.batch_encode_plus(["Tôi đang học ở trường đại học tự nhiên", "Tôi là ai", "Hai tay bưng dĩa í a bánh bò"], return_tensors="pt", padding=True, max_length=512, truncation=True, add_special_tokens=True)
print(inputs2)
print(inputs2['input_ids'].shape)
outputs = model(**inputs2)
print("output", outputs.keys())
print(len(outputs))  # 3

hidden_states = outputs['last_hidden_state']
print(len(hidden_states))  # 13

embedding_output = hidden_states[0]
attention_hidden_states = hidden_states[1:]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing BertModel: ['roberta.encoder.layer.5.output.LayerNorm.bias', 'roberta.encoder.layer.5.attention.output.LayerNorm.bias', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.encoder.layer.9.attention.output.dense.bias', 'roberta.encoder.layer.10.attention.self.key.bias', 'roberta.encoder.layer.10.attention.output.dense.bias', 'roberta.encoder.layer.11.attention.self.key.bias', 'roberta.encoder.layer.2.attention.self.value.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.dense.bias', 'roberta.encoder.layer.4.output.LayerNorm.bias', 'r

{'input_ids': tensor([[    0,   218,    52,   222,    25,   212,  2919,   222,   385,  8249,
             2],
        [    0,   218,     8,   277,     2,     1,     1,     1,     1,     1,
             1],
        [    0,   716,   306, 17591, 18657,  8210,  1517,  1157,  1476,     2,
             1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}
torch.Size([3, 11])
output odict_keys(['last_hidden_state', 'pooler_output'])
2
3


Embedding(64001, 768, padding_idx=1)
Embedding(258, 768)


TypeError: iteration over a 0-d tensor

In [8]:
import re
import joblib
import pandas as pd
import torch
filetrain='train_pharagraph_full.jl'
filetarget='target_strings_full.jl'
train_pharagraph = joblib.load(filetrain)
target_strings = joblib.load(filetarget)

targets = []
for ele in target_strings:
    ele = re.sub(r'[\W_]', ' ', ele)
    ele = re.sub(r'[^\w\s]', '', ele)
    ele = re.sub(r'\t\n', '', ele)
    targets.append(ele)

training_input = []
for ele in train_pharagraph:
    ele = re.sub(r'[\W_]', ' ', ele)
    ele = re.sub(r'[^\w\s]', '', ele)
    ele = re.sub(r'\t\n', '', ele)
    training_input.append(ele)

document = pd.Series(training_input)
summary = pd.Series(targets)

doc_list = []
sum_list = []
for index, doc in enumerate(document):
    if len(doc) < 2000:
        doc_list.append(doc)
        sum_list.append(summary[index])

document = pd.Series(doc_list)
summary = pd.Series(sum_list)



In [10]:
targets = tokenizer(sum_list[:100],  return_tensors="pt", padding=True,max_length=200)
inputs = tokenizer(doc_list[:100],  return_tensors="pt", padding=True,max_length=512)

e:\Code Summary\summarize-torch\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2357: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [14]:
from torch.utils.data import Dataset
import torch
class MyDataset(Dataset):
    def __init__(self, inputs,targets):
        super().__init__()
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])
    
    def __getitem__(self, index):
        inp_input_ids = self.inputs['input_ids'][index]
        inp_token_type_ids = self.inputs['token_type_ids'][index]
        inp_attention_mask = self.inputs['attention_mask'][index]
        tar_input_ids = self.targets['input_ids'][index]
        tar_token_type_ids = self.targets['token_type_ids'][index]
        tar_attention_mask = self.targets['attention_mask'][index]

        return {
            'inp_input_ids' : inp_input_ids,
            'inp_token_type_ids' : inp_token_type_ids,
            'inp_attention_mask' : inp_attention_mask,
            'tar_input_ids' : tar_input_ids,
            'tar_token_type_ids' : tar_token_type_ids,
            'tar_attention_mask' : tar_attention_mask,
        }

In [15]:
dataset = MyDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)


In [26]:
from main import *
from transform import *
word2Topic = joblib.load('word2Topic_v2.jl')
list_topic_count = joblib.load('list_topic_count.jl')

encoder_vocab_size = tokenizer.vocab_size
decoder_vocab_size = tokenizer.vocab_size
transformer = TransformerModel(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
    word2Topic=word2Topic,
    list_topic_count=list_topic_count,
    bert=model
).to(device)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
learning_rate = CustomSchedule(optimizer, d_model)
logging.info(f"learning rate - {learning_rate}")
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: epoch/10)
for batch, row in enumerate(dataloader):
            print(tar_input_ids)
            inp_input_ids,inp_token_type_ids,inp_attention_mask, tar_input_ids = row['inp_input_ids'].to(device), row['inp_token_type_ids'].to(device), row['inp_attention_mask'].to(device), row['tar_input_ids'].to(device)
            train_step( inp_input_ids.to(device), inp_token_type_ids.to(device), inp_attention_mask.to(device), tar_input_ids.to(device) , transformer, optimizer, scheduler)

tensor([    0, 16894,  3139,   963,  2271,    48,    99,  5330, 11603,     6,
           48,   358,   113,    80,     7,  1393,  2075,    24,   273,   123,
          188,  2006, 46779,     2])


AssertionError: Torch not compiled with CUDA enabled

In [8]:
import joblib
import re
import pandas as pd
import torch
from config import *

from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, inputs,targets):
        super().__init__()
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])
    
    def __getitem__(self, index):
        inp_input_ids = self.inputs['input_ids'][index]
        inp_token_type_ids = self.inputs['token_type_ids'][index]
        inp_attention_mask = self.inputs['attention_mask'][index]
        tar_input_ids = self.targets['input_ids'][index]
        tar_token_type_ids = self.targets['token_type_ids'][index]
        tar_attention_mask = self.targets['attention_mask'][index]

        return {
            'inp_input_ids' : inp_input_ids,
            'inp_token_type_ids' : inp_token_type_ids,
            'inp_attention_mask' : inp_attention_mask,
            'tar_input_ids' : tar_input_ids,
            'tar_token_type_ids' : tar_token_type_ids,
            'tar_attention_mask' : tar_attention_mask,
        }

# dataset = MyDataset(inputs,targets)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


In [9]:
import torch
from helper import *
from config import *

dataset = MyDataset(inputs,targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
for batch, inp in enumerate(dataset):
    print(inp.keys())
    break
    

dict_keys(['inp_input_ids', 'inp_token_type_ids', 'inp_attention_mask', 'tar_input_ids', 'tar_token_type_ids', 'tar_attention_mask'])


In [ ]:

train_loss = []
def train_step(inp, tar, transformer, optimizer, scheduler):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    predictions, enc_output, att_weights = transformer(
        inp, tar_inp, 
        True, 
        enc_padding_mask, 
        combined_mask, 
        dec_padding_mask,
    )
    loss = loss_function(tar_real, predictions)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    train_loss.append(loss.item())

In [ ]:
for epoch in range(EPOCHS):
        print("Epoch {}".format(epoch))
        train_loss.clear()
        for batch, row in enumerate(dataset):
            train_step( {key: row[key] for key in row.keys() if 'inp' in key}, {key: row[key] for key in row.keys() if 'tar' in key}, transformer, optimizer, scheduler)
            if batch > 0 and batch % 1000 == 0:
                print('Batch {} Loss {:.4f}'.format(batch, sum(train_loss) / len(train_loss)))
        print('Epoch {} Loss {:.4f}'.format(epoch, sum(train_loss) / len(train_loss)))
        print('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))
        if (epoch > 0 and epoch % 15 == 0):
            torch.save(transformer.state_dict(), f"checkpoints/transformer_epoch_{epoch}.pt")

In [9]:
inputs_1 = torch.randn(999, 11, dtype=torch.float32)
targets_1 = torch.randint(5, (999, 1), dtype=torch.long)
print(inputs_1.shape)
print(targets_1.shape)

torch.Size([999, 11])
torch.Size([999, 1])
